# R-class 3 (2 hours) 

## Deliverable Ex. 1

###  Use tests to answer questions:  
  * Do the have the same mean?
  * Do they have the same variance?
  * Are they compatible with being the same distribution?
  
Ho = both distributions are equal

p(Ho) = p-val=1% => only 1% of the times we will have such a diffence in the means/variances by chance 
                    (diffence ~ real)
p(Ho) = p-val=90% => 90% of the times we will have such a difference in the means/variances by chance 
                    (diff ~not real) 

```
 P(Ho) = *.test$pvalue
 
```

**RANKS:**
    
```
    dat1r <-rank(dat12,ties.method="average")[1:N1]
    dat2r <-rank(dat12,ties.method="average")[(N1+1):N12]
```

In [ ]:
# rank example
#dat1 <- floor(runif(10,0,20))
#dat2 <- floor(runif(10,0,20))
dat1 <- sample(0:50,10)
dat2 <- sample(0:50,10)
cat("dat1=",dat1,"\n")
cat("dat2=",dat2,"\n")

In [ ]:
# previous calculations to compare ranks-by-hand and R-ranks
dat12 <- c(dat1,dat2)
dat12r <- rank(dat12,ties.method="average")
dat1r <- rank(dat12,ties.method="average")[1:10]
dat2r <- rank(dat12,ties.method="average")[11:20]

In [ ]:
dat12s <- sort(dat12)

cat("Doing ranks step by step...\n")
cat("=============================\n")
handmade <- rbind("dat12"=as.character(dat12), dat12s) # as.character only for display purposes
handmade
results <- data.frame("dat1_element"=dat12[1:10], "dat1_positions"=as.character(dat1r), 
                      "dat2_elements"=dat12[11:20], "dat2_positions"=as.character(dat2r))
results

cat("Using R ranks\n")
cat("=============================\n")
all <- rbind("dat12r"=as.character(dat12r)) # as.character only for display purposes
all
all1 <- rbind("dat1r"=as.character(dat1r), "dat2r"=as.character(dat2r))
all1

**TESTS:**

    * USE 99% confidence level

    * F (isher) test: parametric test for the variance
        Use var.test -> use this for parametric tests
        
    * t (Student) test: parametric test for the mean 
        Two cases (sigmas are equal/different - aprox. Welch)
        Use t.test
        
       t.test(x, ..., alternative = ..., var.equal = ..., conf.level = ...)

    
    * Non-parametric test of the mean:
        wilcox.test (test U from Wilcoxon-Mann-Whitney over original data)
        
    * Test F over ranks: Variance tests for large N  
        Use var.test (over ranks)
        
    * Z-test: variance test for large N  
         p(Ho) = 1 - P(HA) = erfc(Z)  
        **Note**: install 'pracma' for erf() 
        
    * K-S test: compare the full distribution
        Use ks.test

# Minimization and model fitting
1. Function roots (unidimensional case):  
   * Polynomial roots: ```polyroot```  
   * Non-polynomial functions roots: ```uniroot```  
2. Maxima and minima searching (unidimensional): ```optimize```  
3. Fit of unidimensional functions to a straight line:  
   * Uni-parametric:   
       - By least squares: ```lm```  
       - Minimizing distances: ```optimize```   (optional)  
   * Multiparametric:``optim``

In [ ]:
library(repr) # resize plots
# Change plot size to 4 x 4
options(repr.plot.width=5, repr.plot.height=5)

## 1. Function roots (unidimensional case)
### a) Polynomial roots: ``polyroot``

1.- $f(x) = 3 + 2 x + x^2 = 0$

2.- $f(x) = (x-3)(x-{1\over2})(x+{1\over2})(x+4) = 3 - {1\over4} x - {49 \over 4} x^2 + x^3 + x^4$  (class ex.)

In [ ]:
#Example 1
# simple way to check if polynomial becomes negative
pol1 <- function(x){  # define polynomial
    res <- 3 + 2*x + x^2
}    
x <- seq(-100,100,by=0.1)
cat("Is it negative?", any(pol1(x) < 0), "\n")   

#root calculation
zeros <- polyroot(c(3,2,1))   # look for roots
print(zeros)                  # no real roots; only complex roots

#plotting
options(repr.plot.width=4, repr.plot.height=4)
curve(pol1, xlim=c(-100,100),cex.axis=0.8, cex.lab=0.8, ylim=c(-1000,10000))
abline(h=0, col="red")

---
---
### CLASS EXERCISE: 

$f(x) = (x-3)(x-{1\over2})(x+{1\over2})(x+4) = 3 - {1\over4} x - {49 \over 4} x^2 + x^3 + x^4$

---
---

### b) Non-poynomial functions: ``uniroot``
* Function values must have opposite signs in both edges of the interval to ensure that at least one root exists in the interval (Bolzano Theorem)

* Search is done in a **given interval** of the unidimensional argument of the function: provide limits!
* It only finds one root

$f(x) = cos(x)-sin(2 x)$


In [ ]:
# define function
fcossin <- function(x) cos(x)-sin(2*x)
# plot it
curve(fcossin, col="red", xlim=c(0,2*pi),lwd=2, ylab="f(x)",
      main=expression(cos(x)-sin(2*x)==0), cex.main=0.8)
grid()
abline(h=0, col="green", lty=2)

# calculate first root in the interval [0,4]
zero <- uniroot(fcossin, lower=0, upper=4)
print(zero)

# plot result
points(zero$root, zero$f.root, pch=16, cex=1.5)


**Warning**: each call to ``uniroot`` only gives the **first** root! You must do as many call as roots you need to find (given the appropriate interval)

---
---
### CLASS EXERCISE: 

   Look for all the roots in 4 intervals

---
---

## 2. Maxima and minima search (unidimensional case)

* If the derivative is easy to calculate, then maxima/minima are the roots of the derivative (derivative=0)
* If the function is not unimodal (unique maximum/minimum) it cannot be assured that the maximum/minimum found is a global one
* If the derivative cannot be analitically calculated: **``optimize``**: ``optimize$maximum``, ``optimize$minimum``, ``optimize$objective`` (we need to define an interval to search)

In [ ]:
options(repr.plot.width=4, repr.plot.height=4)

# Use function above and plot it
interval = c(0,pi)
interval = c(0,3*pi)
curve(fcossin, col="red", xlim=c(0,2*pi), lwd=2, ylab="f(x)",main=expression(cos(x)-sin(2*x)==0))
grid()

# calculate extreme (by default it is a minimum)
extreme <- optimize(fcossin, lower=interval[1], upper=interval[2])
points(extreme$minimum, extreme$objective, pch=16, cex=1.5, col="blue")
cat("Minimum:\n")
print(extreme)

# calculate maximum
extreme <- optimize(fcossin, lower=interval[1], upper=interval[2], maximum=TRUE)
points(extreme$maximum, extreme$objective, pch=16, cex=1.5, col="magenta") # it is not a global maximum
cat("Maximum:\n")
print(extreme)


## 3. Fit of unidimensional functions to a straight line (least squares fit)

### a) Using ``lm``

Let's fit a straight line to a sample of data in the file *hubble_data.txt* (on line platform "Aula Virtual").

The formula to use in **``lm``** can be:

$y \sim x$ for $ y = ax + b$
    
$y \sim x-1$ for $y = ax$

$y \sim 0 + x$ for $y = ax$

$y \sim 1$ for $y = ct$

In addition, errors can be taken into account in the fit (```weight``` = 1/var)

In [ ]:
# read data 
hubble <- read.table("hubble_data.txt", header=TRUE)
str(hubble)

# plot data
plot(hubble, pch=16, col="blue", main="Hubble's data (1929)", 
     xlab="Distance (Mpc)", ylab="Recessional Velocity (km/s)")
grid()

# fit to a straight line using lm(..)
fit_lm <- lm(hubble$recession.velocity ~ hubble$distance-1) 
fit_lm
summary(fit_lm)
# show slope fitted value
print(fit_lm$coefficients)
#hubble$distance
# plot fit line
abline(fit_lm, col="red", lwd=2)

### b) Alternative: Minimizing distances +  ```optimize```
We can have the same result using ``optimize``: create a function (sum of the squares of the distances to the line) that will be minimized and then look for the value of the minimizing slope

In [ ]:
# function to minimize the sum of the distances, squared
fun.sum.residuals <- function(slope,x,y) {
    output <- sum((y-x*slope)^2)
    return(output)
}
# check slopes in [0,2000]
max.slope <- 2000
test.slopes <- numeric(max.slope)
residuals <- numeric(max.slope)
for (i in 1:max.slope) {
    test.slopes[i] <- i
    residuals[i] <- fun.sum.residuals(test.slopes[i], x=hubble$distance, y=hubble$recession.velocity)
}

# plot residuals for every value of the slope
plot(test.slopes, residuals, type="l", col="red", log="y", 
     xlab="slopes", ylab="function to minimize")

# calculate slope that minimize fun.sum.residuals
#        default: maximum=FALSE
#        otther remaining parameters needed (no to be minimized): x,y
opt <- optimize(fun.sum.residuals, lower=0, upper=1000,  x=hubble$distance, y=hubble$recession.velocity)
points(opt$minimum, opt$objective, pch=16, cex=1.5, col="red")
cat("Minimum slope reached at: ")
print(opt$minimum)

# plot legend
legend("topleft",  paste("Res.: Minimum at",round(opt$minimum,2)),
       lty=1,col=c("red","blue"), cex=0.6, bty="n")

---
---
### CLASS EXERCISE: 
Minimize the absolute value of the sum of the distances (instead of the sum of the squares of the distances) and plot the results in the same plot than the least squares minimization.
   

---
---

## 4. Fit to a unidimensional function (multiparametric)
To fit a function with more than 1 parameter, we can use the generic function **```optim```**. Warning! Do not confuse with **```optimize```**.

- Using this function requires having an initial approximated solution. 

- **```optim```** can use different methods to estimate the minimum. By default this is Nelder-Mead (1965), which only requires be able to calculate the function in several points (it is a robust method although it can be slow). More sofisticated methods need an evaluation of the function gradient.

- We will fit $f(x) = p_1 cos(p_2x) + p_2sin(p_1x)$

In [ ]:
#                        PLOT DATA
# read data to be fitted
data <- read.table("dataOptize.txt", header=TRUE)
xdata <- data[,1]
ydata <- data[,2]
# have a look at it
plot(xdata,ydata,pch=16,col="magenta")



In [ ]:
#           PLOT DATA + FUNCTION (with initial guesses)
plot(xdata,ydata,pch=16,col="magenta")
# initial function to be fitted (note that function returns a vector)
funp1p2 <- function(x,p1,p2) {
    f <- p1*cos(p2*x)+p2*sin(p1*x)
    return(f)
}
# some starting values + plot function for this selection
p1_init = 1.0
p2_init = 0.2
xp <- seq(min(xdata),max(xdata),length.out=1000)
yp <- funp1p2(xp,p1_init,p2_init)
lines(xp, yp, col="blue", lty=2)
# also with 'curve'
# curve(funp1p2(x,p1_init,p2_init), add=TRUE, col="blue", lty=2)

In [ ]:
#                 PLOT DATA AND MINIMIZE FUNCTION
#plot(xdata,ydata,pch=16,col="magenta")
#lines(xp, yp, col="blue", lty=2)

## function to be minimized
fun.to.minimize <- function(params,x,y) {
    p1 <- params[1]
    p2 <- params[2]
    output <- sum((y-funp1p2(x,p1,p2))^2)
    #yp1p2 <- funp1p2(x,p1,p2)                  # additional line in green
    #lines(x, yp1p2, col="green", lwd=1) 
    return(output)
}

## using optim(...) to find best fit
solution <- optim(c(p1_init,p2_init), fun.to.minimize, x=xdata, y=ydata)
new.p1 <- solution$par[1]
new.p2 <- solution$par[2]

## plot solution
#yp <- funp1p2(xp,new.p1,new.p2)
#lines(xp, yp, col="red", lwd=2)
#legend("bottomleft",c("initial guess","final fit"),lty=c(2,1),col=c("blue","red"),bty='n',cex=0.8)
#print(solution)


![No image, sorry](optim_movie.gif)

---
---
### CLASS EXERCISE: 

- Fit Hubble data to a line using optim (not adequate - 1 dim-, but possible)

## ---
---
### CLASS EXERCISE: fit to ~Gaussian line

1. Create 40 points between 3.4 and 5.2 following a Gaussian density curve (mean=4.27, sigma=0.34)

2. Move points "y" value a random quantity between -20% and 20% (uniform)

3. Give them an error = sqrt(y)/10

4. Plot points (with error bars) and the "theoretical" initial curve from which points are derived

5. Fit a gaussian minimizing $\chi^2$

6. Add fit plot

In [ ]:
#install.packages("Hmisc")
library(Hmisc)

# Create 40 points between 3.4 and 5.2 following a Gaussian density curve (mean=4.27, sigma=0.34)
xdata <- seq(3.4,5.2,length.out=40)
ydata0 <- dnorm(xdata,mean=4.27,sd=0.34)

# Move points "y" value a random quantity between -20% and 20% (uniform)
rn <- runif(length(xdata),min=-0.2,max=0.2)
ydata <- (1 + rn) * ydata0

erry <- sqrt(ydata)/15
# Plot points (with error bars) and the "theoretical" initial curve from which points are derived
errbar(x=xdata,y=ydata,yplus=ydata+erry, yminus=ydata-erry)
curve(dnorm(x,mean=4.27,sd=0.34), col="green", add=T) # theoretical curve

# 

## Example: Fitting a polynomial to the data with optim

In [ ]:
options(repr.plot.width=4, repr.plot.height=4)
x <- 0:10
y <- c(2,5,4,6,7,9,11,15,20,25,30)

plot(x,y)

In [ ]:
fun.to.fit <- function(x, a, b,c){
    a + b*x + c*x^2 
}

fun.to.minimize <- function(coefs,x,y){
    a <- coefs[1]
    b <- coefs[2]
    c <- coefs[3]
    out <- sum((y-fun.to.fit(x,a,b,c))^2)
}

In [ ]:
a0 <- 1
b0 <- 1
c0 <- 1
fit_optim <- optim(c(a0,b0,c0), fun.to.minimize, x=x, y=y)
afit <- fit_optim$par[1]
bfit <- fit_optim$par[2]
cfit <- fit_optim$par[3]
fit_optim$par
fit_lm <- lm(y~poly(x,2,raw=TRUE)) 
fit_lm$coefficients
help(optim)

In [ ]:
plot(x,y)
curve(fun.to.fit(x,afit,bfit,cfit),add=TRUE, col='blue',lty=2)
lines(x,predict(fit_lm,data.frame(x=x)),col="red", lty=6)
legend("topleft", c("OPTIM", "LM"), lty=c(2,6), col=c("blue","red"), cex=0.8, bty='n')